# Identifying Phase Transitions

In this notebook, we will use numerous methods to characterize the phase behavior of systems of hard polygons.
At low pressures, systems will be in a fluid phase.
At high enough pressures, the system will form a solid.
There is also an intermediate hexatic phase, but that takes more care and much larger simulations to identify - see [ Shape and Symmetry Determine Two-Dimensional Melting Transitions of Hard Regular Polygons](http://dx.doi.org/10.1103/PhysRevX.7.021001).

## Select a state point

As a first step, we will identify the state (fluid or solid), for a particular combination of parameters after the system has equilibrated.
We will perform various analyses using [Freud](glotzerlab.engin.umich.edu/freud) to show how we can identify the phase.
Once we have identified that this occurs for a some pressure for a given shape, we will look at multiple data points in concert to identify the critical point.

In [ ]:
import signac

project = signac.get_project()

In [ ]:
import sys
from ipywidgets import interact

JOB = None

schema = project.detect_schema()
ns = schema['n'][int]
betaPs = schema['betaP'][float]
seeds = schema['seed'][int]

@interact(n=ns, betaP=betaPs, seed=seeds)
def select(n=5, betaP=13.2, seed=0):
    global JOB
    jobs = project.find_jobs(dict(n=n, betaP=betaP, seed=seed))
    if len(jobs) == 0:
        print("No jobs found for this selection.", file=sys.stderr)
    elif len(jobs) > 1:
        print("Multiple jobs matched this selection.", file=sys.stderr)
    else:
        JOB = list(jobs)[0]
        print("Selected", JOB)

Use the widget above to select a specific state point to examine.
Then, run (or re-run) the below cells to examine the results from that simulation.

## Equilibration

As a first step, we plot the average hexatic order parameter and system density over time. 
Over the course of the simulation, these quantities will stabilize as the system equilibrates.

The [k-atic order parameter](https://freud.readthedocs.io/en/stable/order.html#freud.order.HexOrderParameter) measures *k-fold* order in a system of particles by calculating the angles between a particle and its neighbors.
Here, we calculate $\psi$, the magnitude of the average 6-atic, or **hexatic**, order parameter, which measures the 6-fold order.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import gsd.hoomd
%matplotlib inline

with gsd.hoomd.open(JOB.fn('trajectory.gsd')) as traj:
    N = traj[-1].particles.N
# alternately, use a custom log operation to log phi directly

log = np.genfromtxt(fname=JOB.fn('log.dat'), names=True)
psi = np.load(JOB.fn('order.npz'))

fig, ax = plt.subplots(figsize=(4, 2.2), dpi=140)
ax2 = ax.twinx()

ax.plot(log['timestep'], N * JOB.doc.poly_area / log['volume'])
ax.set_xlabel('time step')
ax.set_ylabel('Packing Fraction')

ax2.plot(psi['steps'], np.absolute(psi['psi'].mean(axis=1)), color='red')
ax2.set_ylabel('Hexatic Order Parameter ($\psi$)', color='red')

plt.show()

Fluids and solids will show different behavior:
* **Fluid:** Low pressure fluids will exhibit very little hexatic order, the equilibrium $\psi$ will be very small.
* **Solid:** In ordered solids, $\psi$ will be large, approximately 0.5 and greater.

# Hexatic order parameter

We can also visualize the system directly, with each particle colored by the value of the local hexatic order parameter.
Note that instead of using the stored values, we are now using [`freud`](https://freud.readthedocs.io/en/stable/order.html#freud.order.HexOrderParameter) to calculate the order parameter on the fly each time we visualize a new frame.

In [ ]:
import freud
from draw_utils import quat2ang, draw_config, draw_pmft, draw_voronoi

with gsd.hoomd.open(JOB.fn('trajectory.gsd')) as traj:
    num_frames = len(traj)-1 

In [ ]:
@interact(frame=(1, num_frames))
def frame_demo(frame=num_frames):
    fix, ax = plt.subplots(1, 1, figsize=(8, 8))
    with gsd.hoomd.open(JOB.fn('trajectory.gsd')) as traj:
        frame = traj[frame]
        
        box = freud.box.Box.from_box(frame.configuration.box[:2].tolist())
        hop = freud.order.HexOrderParameter(rmax=1.2, k=6)
        hop.compute(box, frame.particles.position)
        
        draw_config(fig, ax, box, frame.particles.position, quat2ang(frame.particles.orientation), hop.psi, JOB.sp.n)

Fluids and solids will show different behavior:
* **Fluid:** Fluids have very little correlation in hexatic order. Particles will show the full range of the color wheel:
* **Solid:** In ordered solids, all the local environments are ordered. Particles will most all be the same color (except in a few local regions near defects).

## RDF

Another way we can see this more clearly is using the [radial distribution function (RDF)](https://en.wikipedia.org/wiki/Radial_distribution_function), which measures the system density as a function of distance from a particle.
The RDF is measured by calculating the average number of particles at a given distance from each particle and then averaging that measure over all particles.
In a perfect crystal, the RDF should be just a set of [delta functions](https://en.wikipedia.org/wiki/Delta_function) since all particles are located at precise fixed distances from one another.
In systems of particles like the ones we are simulating, we instead expect to see smoother sets of peaks.

Below, we calculate the RDF by binning all particles using `freud` (see [the documentation](https://freud.readthedocs.io/en/stable/density.html#freud.density.RDF) for more information).

In [ ]:
@interact(frame=(1, num_frames))
def rdf_demo(frame=num_frames):
    with gsd.hoomd.open(JOB.fn('trajectory.gsd')) as traj:
        frame = traj[frame]
            
        box = freud.box.Box.from_box(frame.configuration.box[:2].tolist())
        rdf = freud.density.RDF(np.sqrt(box.Lx**2 + box.Ly**2)/5, box.Lx/1000)
        rdf.compute(box, frame.particles.position)

    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    ax.plot(rdf.R, rdf.RDF)

Both fluids and solids will show a few strong peaks as there are short range correlations. Where they differ:
* **Fluid:** g(r) will tend toward 1.0 at moderate and large values of r.
* **Solid:** g(r) will show oscillations out to large values of r. Solids take some time to develop, so early frames may show fluid-like behavior while later frames develop longer range correlations.

However, as it is a function of distance alone, the RDF is clearly a limited way of characterizing our system, which we know should exhibit some ordering that depends on the angle between particles.

## PMFT

To get a more informative picture, we can look at the potential of mean force and torque (PMFT).
The PMFT is a generalization of the classical [potential of mean force](https://en.wikipedia.org/wiki/Potential_of_mean_force) (PMF), which measures the average potential energy surface about each particle in the system as a function of distance.
The PMF can be calculated directly as $w(r) = -\beta \log(g(r))$ (where $g(r)$ is the radial distribution function).

The PMFT takes this one step further, looking at the potential energy surface as a function of both distance and angle.
As a result, the PMFT can capture the differences in a potential energy surface induced by, for instance, the shape of a particle.
In practice, the PMFT is calculated by [binning space and counting particles](https://freud.readthedocs.io/en/stable/pmft.html#freud.pmft.PMFTXY2D).
We plot the results below.

In [ ]:
@interact(frame=(1, num_frames))
def pmft_demo(frame=num_frames):
    with gsd.hoomd.open(JOB.fn('trajectory.gsd')) as traj:
        frame = traj[frame]
            
        box = freud.box.Box.from_box(frame.configuration.box[:2].tolist())
        pmft = freud.pmft.PMFTXY2D(4, 4, 300, 300)
        pmft.compute(box, frame.particles.position, quat2ang(frame.particles.orientation))

    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    draw_pmft(fig, ax, pmft, JOB.sp.n)

Looking at early frames of our simulation, we see that the PMFT looks almost circularly uniform.
In fact, in this case the PMFT shows the same information as the RDF.
There is a low energy ring immediately surrounding the central particle, and past that we see a mostly uniform potential energy surface at all distances.
The PMFT doesn't tell us much about the fluid-solid transition, but does help explain how the effective interactions between polygons changes.

* **Low vertex polygons**: In later frames, however, we start to see more features.
In particular, the PMFT shows smaller energy wells near the edges of the polygon, indicating polygons like to sit edge to edge. 
* **High vertex polygons**: In later frames, you should still see circular rings. These polygons are close enough to circles that they don't feel there edges strongly.

Note: The system sizes in this tutorial are very small and the PMFTs are only obtained from a single frame. For much cleaner PMFT data, one should use larger system sizes and average over many frames. See the paper [Shape and Symmetry Determine Two-Dimensional Melting Transitions of Hard Regular Polygons](http://dx.doi.org/10.1103/PhysRevX.7.021001) for well-sampled PMFTs of polygons.

This is additional information not present in the orientationally averaged RDF.

## Voronoi diagram

We saw defects in the system with the hexatic order parameter earlier.
Another way to count defects is to look at the [Voronoi diagram](https://en.wikipedia.org/wiki/Voronoi_diagram) of the system.
Using the particle positions as reference points, the Voronoi diagram partitions space and finds every point is associated with the closest reference point.
In a **perfectly ordered system**, we would expect the Voronoi diagram to divide space into perfect hexagons.
Anywhere there are defects in the system, however, we would expect the Voronoi diagram to show pentagons and heptagons (and occasionally larger polygons).
To demonstrate this, we plot the Voronoi diagrams below, with each Voronoi polygon colored by the number of sides.

In [ ]:
@interact(frame=(1, num_frames))
def voronoi_demo(frame=num_frames):
    with gsd.hoomd.open(JOB.fn('trajectory.gsd')) as traj:
        frame = traj[frame]
            
        box = freud.box.Box.from_box(frame.configuration.box[:2].tolist())
        voronoi = freud.voronoi.Voronoi(box, np.sqrt(box.Lx**2 + box.Ly**2)/5)
        voronoi.compute(frame.particles.position, box)

    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    draw_voronoi(fig, ax, box, voronoi.polytopes)

As we see, early on in the simulation that there are many 5- and 7-gons, as the system is still equilibrating.
When equilibrated, you should see:

* **Fluid**: Mostly hexagons, but a high concentration of 5- and 7-gons connected to each other.
* **Solid**: Mostly hexagons, with a few 5- and 7-gons connected to each other.

Solids in 2D only have "quasi" long range order due to an appreciable concentration of defects at equilibrium. Compare the Voronoi diagram to the same frame above with the hexatic order parameter coloring and the defects should be visible in the same locations.

## Identify fluid-solid transition

Finally, we return to our original problem: finding the pressure at which the system undergoes a phase transition.
We plot the average value of the hexatic order parameter (in the final $N$ frames of our simulation) as a function of pressure, and we look for a pressure at which we see a sudden jump in the order parameter.

In [ ]:
@interact(n=project.detect_schema()['n'][int], num_frames=(1, 10))
def transition_demo(n, num_frames=5):
    avg_hop = {}
    for betaP, group in project.find_jobs({"n": n}).groupby('betaP'):
        hops = []
        for job in group:
            with gsd.hoomd.open(job.fn('trajectory.gsd')) as traj:
                for frame in traj[-num_frames:]:
                    box = freud.box.Box.from_box(frame.configuration.box[:2].tolist())
                    hop = freud.order.HexOrderParameter(rmax=1.2, k=6)
                    hop.compute(box, frame.particles.position)
                    hops.append(np.absolute(np.mean(hop.psi)))
        avg_hop[betaP] = np.mean(hops)
    fig, ax = plt.subplots(1, 1)
    ax.plot(avg_hop.keys(), avg_hop.values(), 's')